In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install gradio

In [9]:
from huggingface_hub import login
login()

In [11]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gradio as gr

class ArtEmotionChatbot:
    def __init__(self, model_path, llama_model_name):
        """
        Initialize the chatbot with your trained Keras model and LLaMA 2.
        """
        # Load the Keras model
        self.model = load_model(model_path)
        self.img_size = (180, 180)  # Image size used for training

        # Initialize LLaMA 2
        self.tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
        self.llama_model = AutoModelForCausalLM.from_pretrained(
            llama_model_name, torch_dtype="auto", device_map="auto"
        )
        self.chat_pipeline = pipeline("text-generation", model=self.llama_model, tokenizer=self.tokenizer)

        # Define emotion labels
        self.emotion_labels = ['positive', 'negative', 'mixed']  # Adjust based on your model

    def preprocess_image(self, image):
        """
        Preprocess the uploaded image for prediction.
        """
        img = image.resize(self.img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array

    def predict_emotion(self, image):
        """
        Predict the emotion based on the uploaded image.
        """
        img_array = self.preprocess_image(image)
        predictions = self.model.predict(img_array)
        emotion_index = np.argmax(predictions)
        return self.emotion_labels[emotion_index]

    def generate_response(self, emotion, user_input):
        """
        Generate chatbot response based on emotion and user input.
        """
        if "emotion" in user_input.lower():
            # If user asks about the emotion, generate response based on the predicted emotion
            response = f"The painting evokes a {emotion} feeling. It might make one think of peace, joy, or serenity if it's positive, or other feelings depending on the mood."
        else:
            # Default behavior for other questions
            prompt = f"The user seems {emotion}. Respond empathetically to their input:\nUser: {user_input}\nChatbot:"
            response = self.chat_pipeline(prompt, max_length=200, temperature=0.7)
            response = response[0]["generated_text"].split("Chatbot:")[-1].strip()

        return response

# Instantiate the chatbot
!wget https://drive.google.com/file/d/1B2cW5le9tBE7LaqqqaYtnGcHgBzybKOO/view?usp=drive_link
model_path = "./emotion_model.keras"
llama_model_name = "kl3658/emotion_Llama-finetune"  # Hugging Face model
chatbot = ArtEmotionChatbot(model_path, llama_model_name)

# Define global variables to store predicted emotion
predicted_emotion = None

# Gradio function for analyzing the image
def analyze_image(image):
    global predicted_emotion
    try:
        predicted_emotion = chatbot.predict_emotion(image)
        return f"The predicted emotion is: {predicted_emotion}"
    except Exception as e:
        return f"Error: {e}"

# Gradio function for chatbot conversation
def chat_with_bot(user_input):
    global predicted_emotion
    if not predicted_emotion:
        return "Please analyze an image first by clicking 'Analyze'."
    response = chatbot.generate_response(predicted_emotion, user_input)
    return response

# Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("## 🎨 Image Emotion Analysis & Chatbot")

    # Image upload and analysis section
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload an Image")
            analyze_button = gr.Button("Analyze")
        with gr.Column():
            analysis_output = gr.Textbox(label="Analysis Result")

    analyze_button.click(analyze_image, inputs=[image_input], outputs=[analysis_output])

    # Chatbot section
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(lines=2, placeholder="Type your question here...", label="Chat with the bot")
            chat_button = gr.Button("Send")
        with gr.Column():
            chat_output = gr.Textbox(label="Chatbot Response")

    chat_button.click(chat_with_bot, inputs=[user_input], outputs=[chat_output])

# Launch the interface
interface.launch()

--2024-12-21 04:39:51--  https://drive.google.com/file/d/1B2cW5le9tBE7LaqqqaYtnGcHgBzybKOO/view?usp=drive_link
Resolving drive.google.com (drive.google.com)... 108.177.119.101, 108.177.119.138, 108.177.119.102, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.101|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized

Username/Password Authentication Failed.


ValueError: File not found: filepath=./emotion_model.keras. Please ensure the file is an accessible `.keras` zip file.